In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

COUNTRIES = ['France', 'Germany', 'South Korea', 'Japan', 'Russia']
SEQ_LEN = 10
FUTURE_PERIOD_PREDICT = 1
MODEL_PARAMS = {
    'batch_size': 128,
    'epochs': 1000,
    'patience': 10,
    'reduce_lr_patience': 5
}

def load_and_preprocess_data(country):
    test_path = f"{country}/{country}.csv"
    test_df = pd.read_csv(test_path, parse_dates=["Date"], index_col=[0])
    train_path = "weighted_data.csv"
    train_df = pd.read_csv(train_path, index_col=[0])
    scaler = MinMaxScaler()
    test_scaled = scaler.fit_transform(test_df[-215:])
    return train_df.values, test_scaled, scaler

def create_sequences(data):
    X, Y = [], []
    for i in range(SEQ_LEN, len(data) - FUTURE_PERIOD_PREDICT + 1):
        X.append(data[i - SEQ_LEN:i, :18])
        Y.append(data[i + (FUTURE_PERIOD_PREDICT - 1), 18])
    return np.array(X), np.array(Y)

def build_model():
    model = Sequential()
    model.add(Bidirectional(GRU(64, return_sequences=False), input_shape=(SEQ_LEN, 18)))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation="tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="relu"))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_callbacks(country):
    return [
        EarlyStopping(monitor='val_loss', patience=MODEL_PARAMS['patience'],
                     verbose=1, mode='min', restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.01,
                         patience=MODEL_PARAMS['reduce_lr_patience'], mode='auto'),
        ModelCheckpoint(
            filepath=f'models/{country}_model.h5',
            monitor='val_loss',
            verbose=1,
            save_weights_only=True,
            mode='min'
        )
    ]

def evaluate_model(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    pred_copies = np.repeat(predictions, 19, axis=-1)
    pred = scaler.inverse_transform(np.reshape(pred_copies, (len(predictions), 19)))[:, 0]
    actual_copies = np.repeat(y_test, 19, axis=-1)
    actual = scaler.inverse_transform(np.reshape(actual_copies, (len(y_test), 19)))[:, 18]
    metrics = {
        'mae': mean_absolute_error(pred, actual),
        'mape': mean_absolute_percentage_error(pred, actual),
        'r2': r2_score(y_true=actual, y_pred=pred)
    }
    return metrics, pred, actual

def process_country(country):
    print(f"\nProcessing {country}...")
    train_data, test_data, scaler = load_and_preprocess_data(country)
    X_train, y_train = create_sequences(train_data)
    X_test, y_test = create_sequences(test_data)
    model = KerasRegressor(build_fn=build_model, verbose=1)
    callbacks = create_callbacks(country)
    model.fit(
        X_train, y_train,
        batch_size=MODEL_PARAMS['batch_size'],
        epochs=MODEL_PARAMS['epochs'],
        validation_data=(X_test, y_test),
        callbacks=callbacks
    )
    metrics, predictions, actuals = evaluate_model(model, X_test, y_test, scaler)
    print(f"\nМетрики для {country}:")
    for name, value in metrics.items():
        print(f"{name.upper()}: {value:.4f}")
    return metrics, predictions, actuals

def main():
    results = {}
    for country in COUNTRIES:
        metrics, pred, actual = process_country(country)
        results[country] = {
            'metrics': metrics,
            'predictions': pred,
            'actuals': actual
        }
    return results

if __name__ == "__main__":
    all_results = main()